# Assignment 3: Visualization with Plotly

<b>Course</b>: CS661A - Big Data Visual Analytics<br>
<b>Instructor</b>: Dr. Soumya Dutta<br>
<br>
<b>Name</b>: Kaushik Raj V Nadar<br>
<b>Roll No</b>: 200499<br>
<b>Department</b>: Biological Sciences \& Bioengineering<br>
<b>Date</b>: 19/03/2023<br>
<br>
<b>Environment</b>: Python 3.8.10 and Jupyter notebook<br>

Importing the required libraries

In [3]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import vtk

Load the data

In [4]:
# Load the vti file
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("Data/mixture.vti")
reader.Update()

# Get the vtkImageData object
data = reader.GetOutput()

# Get the dimensions of the data
dims = data.GetDimensions()

# Access the scalar values of the data
scalar_values = data.GetPointData().GetScalars()

# Define the 3D dataset
x, y, z = np.mgrid[:dims[0], :dims[1], :dims[2]]
numpy_data = np.array(scalar_values)
numpy_data = numpy_data.reshape(dims[2], dims[1], dims[0])
numpy_data = numpy_data.transpose(2,1,0)
values = numpy_data.flatten()

Create the Isosurface Plot

In [5]:
# Define the isosurface
iso_fig = go.FigureWidget(data=go.Isosurface(       
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        isomin = 0,
        isomax = 0,
        surface_count=1,    
        value=values,
        colorbar=dict(thickness=20),    
        colorscale="plasma",    
        cmin=values.min(),  
        cmax=values.max(),      
        showscale=False # remove colorbar
    )
)

# Define the layout of the isosurface
iso_fig.update_layout(
    title='Isosurface',
    scene=dict(     
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
    )
)

FigureWidget({
    'data': [{'cmax': 0.43280163,
              'cmin': -0.99355406,
              'colorbar': {'thickness': 20},
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'isomax': 0,
              'isomin': 0,
              'showscale': False,
              'surface': {'count': 1},
              'type': 'isosurface',
              'uid': 'f3222557-bb5b-45c5-ac96-aed80818794d',
              'value': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.9426848 , -0.8584828 ,
                              -0.72851294], dtype=float32),
            

Create the Histogram Plot

In [ ]:
# Define the histogram
hist_fig = go.FigureWidget(
    data = go.Histogram(
        x=values,
        nbinsx = 30,
        name='Histogram',
        marker_color='rgb(101, 109, 242)',
    )
)
    
# Define the layout of the histogram
hist_fig.update_layout(
    title_text='Histogram',
    xaxis_title='Vortex scalar values',
    yaxis_title='Frequency',
    showlegend=False
)

Create a Slider for adjusting the isovalue

In [ ]:
# Define the slider
slider = widgets.FloatSlider(
    min=values.min(), 
    max=values.max(), 
    step=0.01, 
    value=0.0, 
    description='Isovalue:',
    continuous_update=False,
    orientation='horizontal',
    layout=widgets.Layout(width='50%')
)

# Define the callback function
def update_plot(change):
    '''Update the plot when the slider value changes.'''

    # Update the isosurface
    with iso_fig.batch_update():
        iso_fig.data[0].update(
            isomin = change['new'],
            isomax = change['new'],
            value = values
        )

    # Update the histogram
    with hist_fig.batch_update():
        hist_fig.data[0].update(
            x = values[np.where((values >= change['new']-0.25) & (values <= change['new']+0.25))],
            nbinsx = 27
        )

# Link the slider to the callback function
slider.observe(update_plot, 'value')

Create a Reset button to reset the isovalue to the default value(=0.0)

In [ ]:
# Define the reset button
button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='info', 
    tooltip='Reset',
    icon='refresh' # (FontAwesome names without the `fa-` prefix)
)

# Define the reset function
def reset_plot(button):
    '''Reset the plot to the initial state.'''

    slider.disabled = True      # Disable the slider to prevent the callback function from being called
    slider.value = 0.0      # Reset the slider value

    # Reset the isosurface
    with iso_fig.batch_update():
        iso_fig.data[0].update(
            isomin = 0.0,
            isomax = 0.0,
            value = values
        )
    
    # Reset the histogram
    with hist_fig.batch_update():
        hist_fig.data[0].update(
            x = values, 
            nbinsx = 30
        )

    slider.disabled = False     # Enable the slider again
    
# Link the button to the reset function
button.on_click(reset_plot)

Display the interactive interface

In [ ]:
# Display the widgets
hbox1 = widgets.HBox([slider, button])      # Put the slider and button in a horizontal box
hbox2 = widgets.HBox([iso_fig, hist_fig])   # Put the isosurface and histogram in a horizontal box
main_box = widgets.VBox([hbox1, hbox2])         # Put the horizontal boxes in a vertical box
display(main_box)                           # Display the main box